In [3]:
import cv2
import os
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import PIL

import torchvision
from torchvision import datasets, transforms

from torch.utils.data import DataLoader, TensorDataset

In [4]:
dataset = datasets.ImageFolder('./coronahack-chest-xraydataset/dataset',
                               transform=transforms.Compose([transforms.Resize((256,256)),
                                   np.array, torch.tensor, lambda x: torch.reshape(x,(3,256,256))/255]))
trainset, valset = torch.utils.data.random_split(dataset, [5000, 284])

loaders = {'train': torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True),
           'test':  torch.utils.data.DataLoader(valset)}

In [5]:
dataset.class_to_idx

{'bacteria': 0, 'normal': 1, 'viral': 2}

In [6]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=(4,4), stride=(2,2), padding=(1,1), bias=False)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=(4,4), stride=(2,2), padding=(1,1), bias=False)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=(4,4), stride=(2,2), padding=(1,1), bias=False)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=(4,4), stride=(2,2), padding=(1,1), bias=False)
        self.batch1 = nn.BatchNorm2d(64)
        self.batch2 = nn.BatchNorm2d(64)
        self.batch3 = nn.BatchNorm2d(128)
        self.batch4 = nn.BatchNorm2d(256)
        self.flatten = nn.Flatten()
        self.dense = nn.Linear(65536, 3)

    def forward(self, x):
        """
        Forward-propogate through the neural net.

        Apply rounds of convolution, then batch normalization, leaky-relu, and dropout.
        """
        x = self.conv1(x)
        x = self.batch1(x)
        x = F.leaky_relu(x, negative_slope=0.20)
        x = F.dropout(x, 0.5)

        x = self.conv2(x)
        x = self.batch2(x)
        x = F.leaky_relu(x, negative_slope=0.20)
        x = F.dropout(x, 0.4)

        x = self.conv3(x)
        x = self.batch3(x)
        x = F.leaky_relu(x, negative_slope=0.20)
        x = F.dropout(x, 0.4)

        x = self.conv4(x)
        x = self.batch4(x)
        x = F.leaky_relu(x, negative_slope=0.20)
        x = F.dropout(x, 0.4)

        x = self.flatten(x)
        x = self.dense(x)
        return F.log_softmax(x, dim=1)

In [7]:
model = Model()

In [8]:
example = loaders['train'].dataset[8][0].reshape(1,3,256,256)

In [ ]:
model(example)